In [1]:
import os
os.environ["KERAS_BACKEND"] ='tensorflow'
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm.auto import tqdm
tqdm.pandas()
import keras 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from keras import layers, Model, Input
from keras_hub.src.models.llama.llama_decoder import LlamaTransformerDecoder
from keras_hub.src.models.llama.llama_layernorm import LlamaLayerNorm

/home/mhossai5/.conda/envs/knlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-04 14:53:16.482746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764881596.495679  118629 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764881596.499885  118629 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data Loading

In [2]:
import h5py
DATA_PATH = '../RLEAAI/data/HIV/'
loaded_data = {}
with h5py.File(os.path.join(DATA_PATH,"HIV-RLEAAI-ProtT5-Full.h5"), 'r') as hf:
    for seq in hf.keys():
        loaded_data[seq] = hf[seq][:]

In [3]:
df = pd.read_excel("https://github.com/zhouyu9931/RLEAAI/raw/refs/heads/main/data/dataset_hiv.xlsx")
df.head()

,antibody_seq,virus_seq,label,split
0,QMKLMQSGGVMVRPGESATLSCVASGFDFSRNGFEWLRQGPGKGLQ...,MRVMGIRKNYQHLWREGILLLGILMICSAADNLWVTVYYGVPVWRE...,0,seen
1,QPQLQESGPGLVEASETLSLTCTVSGDSTGRCNYFWGWVRQPPGKG...,MRVRGIPRNWPQWWIWGILGFWMIIICRVVGNMWVTVYYGVPVWTD...,0,seen
2,QVQLLQSGAAVTKPGASVRVSCEASGYNIRDYFIHWWRQAPGQGLQ...,MRVMEIQRNCQHWWIWGILGFWMLMICNVRGWWVTVYYGVPVWKEA...,1,seen
3,QSQLQESGPRLVEASETLSLTCNVSGESTGACTYFWGWVRQAPGKG...,MRVKETQMNWPNLWKLGTLILGLVIICSASXNLWVTVYYGVPVWRD...,1,seen
4,QEQLVESGGGVVQPGGSLRLSCLASGFTFHKYGMHWVRQAPGKGLE...,MRVTGTQRNCQQWWIWIWIILGFWWMLMMCKGEKLWVTVYYGVPVW...,1,seen


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

def process_sequence_tf(x_emb, max_len=1024, pad_value=0.0):
    seq_len = x_emb.shape[0]
    
    if seq_len > max_len:
        # truncate
        x_emb = tf.convert_to_tensor(x_emb[:max_len])
        mask = tf.ones([max_len], dtype=tf.float32)
    else:
        # pad
        pad_len = max_len - seq_len
        paddings = [[0, pad_len], [0, 0]]
        x_emb = tf.pad(x_emb, paddings, constant_values=pad_value)
        mask = tf.pad(tf.ones([seq_len], dtype=tf.float32), [[0, pad_len]], constant_values=0.0)
    
    return x_emb, mask

# -----------------------------
# Keras Sequence Loader
# -----------------------------
class DataSequenceLoader(Sequence):
    def __init__(self, df, batch_size=32, shuffle=True, max_len=512, pad_value=0.0):
        self.x1_emb = df["antibody_seq"].values
        self.x2_emb = df["virus_seq"].values
        self.labels = df["label"].values
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.max_len = max_len
        self.pad_value = pad_value
        self.indices = np.arange(len(df))
        if shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, idx):
        batch_idx = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        x1_list, x2_list, m1_list, m2_list = [], [], [], []
        labels_list = []

        for i in batch_idx:
            emb1 = loaded_data[self.x1_emb[i]].reshape(-1, 1024)
            emb2 = loaded_data[self.x2_emb[i]].reshape(-1, 1024)

            # ---- process embeddings + mask ----
            x1_pad, mask1 = process_sequence_tf(emb1, max_len=self.max_len, pad_value=self.pad_value)
            x2_pad, mask2 = process_sequence_tf(emb2, max_len=self.max_len, pad_value=self.pad_value)

            x1_list.append(x1_pad.numpy())
            x2_list.append(x2_pad.numpy())
            m1_list.append(mask1.numpy())
            m2_list.append(mask2.numpy())
            labels_list.append(self.labels[i])

        # Convert lists to arrays for batch
        x1_batch = np.stack(x1_list, axis=0)
        x2_batch = np.stack(x2_list, axis=0)
        m1_batch = np.stack(m1_list, axis=0)
        m2_batch = np.stack(m2_list, axis=0)
        labels_batch = np.array(labels_list)

        return (x1_batch, x2_batch, m1_batch, m2_batch), labels_batch


In [5]:
# loader  = DataSequenceLoader(df,batch_size=4,shuffle =True)
# for x in loader:
#     break

In [6]:
# -------------------------------------------------------------------
# Hybrid Pooling Layer (Max + Avg)
# -------------------------------------------------------------------
class HybridPooling(layers.Layer):
    def call(self, x):
        max_pooled = keras.ops.max(x, axis=1)
        avg_pooled = keras.ops.mean(x, axis=1)
        return keras.ops.concatenate([max_pooled, avg_pooled], axis=-1)


# -------------------------------------------------------------------
# Conv Block (Conv1D → ReLU → Dropout → MaxPool)
# -------------------------------------------------------------------
def conv_block(x, filters =  100, kernel_sz =20, stride =10, dropout = 0.5):
    x = layers.Conv1D(filters, kernel_sz, strides=stride)(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.MaxPooling1D(pool_size=3, strides=1, padding="same")(x)
    return x


# -------------------------------------------------------------------
# Cross-attention (Query=X1, Key=X2, Value=X2)
# -------------------------------------------------------------------
def cross_attention_block(query, key, value, mask, num_heads =4, key_dim =32):
    attn = layers.MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=key_dim,
        dropout=0.0,
        kernel_initializer="glorot_uniform",
        use_bias=True,
        # flash_attention=None,  # if GPU supports
    )(query, key, value, key_mask=mask, value_mask=mask)
    return attn


# -------------------------------------------------------------------
# LLaMA Self-Attention Block
# -------------------------------------------------------------------
def llama_self_attention(x, mask, hidden_dim =100, num_heads = 4):
    # single LLaMA decoder layer
    llama = LlamaTransformerDecoder(
        intermediate_dim=hidden_dim * 4,
        # num_heads=num_heads,
        num_query_heads=8,
        num_key_value_heads=2,
        dropout=0.0,
        layer_norm_epsilon=1e-5,
        activation="silu"
    )
    return llama(x, decoder_padding_mask=mask)

# -------------------------------------------------------------------
# Functional API Model (PyTorch → Keras Conversion)
# -------------------------------------------------------------------
def build_model(
    input_dim=1024,
    conv_out=100,
    kernel_sz=20,
    stride=10,
    heads=4,
    d_dim=32,
    drop_pool=0.4,
    drop_linear=0.4
):
    # Inputs
    inp1 = Input((None, 1024))
    inp2 = Input((None, 1024))
    mask1 = Input((None, ))
    mask2 = Input((None, ))

    # ---------------------------------------------------
    # 1) Convolutional features
    # ---------------------------------------------------
    # p1 = conv_block(inp1, conv_out, kernel_sz, stride, drop_pool)
    # p2 = conv_block(inp2, conv_out, kernel_sz, stride, drop_pool)
    x_dim = 384
    p1 = layers.Dense(x_dim, use_bias=False,name='stem1')(inp1)
    p1 = layers.BatchNormalization(momentum=0.95,name='bn1')(p1)
    
    p2 = layers.Dense(x_dim, use_bias=False,name='stem2')(inp2)
    p2 = layers.BatchNormalization(momentum=0.95,name='bn2')(p2)
    
    # ---------------------------------------------------
    # 2) Self Attention using LLaMA blocks
    # ---------------------------------------------------
    s1 = llama_self_attention(p1, mask1, conv_out, heads)
    s2 = llama_self_attention(p2, mask2, conv_out, heads)

    # ---------------------------------------------------
    # 3) Cross Attention (1→2 and 2→1)
    # ---------------------------------------------------
    c1 = cross_attention_block(p1, p2, p2, mask2, heads, d_dim)
    c2 = cross_attention_block(p2, p1, p1, mask1, heads, d_dim)

    # Add residual (same as PyTorch + skip)
    sc1 = layers.Add()([s1, c1])
    sc2 = layers.Add()([s2, c2])

    sc1 = layers.Dropout(drop_pool)(sc1)
    sc2 = layers.Dropout(drop_pool)(sc2)

    # ---------------------------------------------------
    # 4) Hybrid Pooling (max + mean)
    # ---------------------------------------------------
    h1 = HybridPooling()(sc1)
    h2 = HybridPooling()(sc2)

    merged = layers.Concatenate()([h1, h2])

    # ---------------------------------------------------
    # 5) MLP Head
    # ---------------------------------------------------
    x = layers.Dense(256, activation="relu")(merged)
    x = layers.Dropout(drop_linear)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(drop_linear)(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    return Model(inputs=[inp1, inp2, mask1, mask2], outputs=out)

In [13]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score, roc_auc_score,
    average_precision_score, matthews_corrcoef, confusion_matrix
)

# ================================================================
# Setup
# ================================================================
train_df,unseen_df = df[df['split']=='seen'].copy(),  df[df['split']=='unseen'].copy()

X = train_df.index.values  # DataSequenceLoader loads using df, so features = df itself
y = train_df['label'].values
bs =32
unseen_loader  = DataSequenceLoader(unseen_df, batch_size=bs, shuffle=False)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_results = []


# ================================================================
# 5-Fold Training Loop
# ================================================================
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    WEIGHT_PATH = f"../weights/LlamaCrossAttn_HIV-RLEAAI/model_fold_{fold}.weights.h5"
    os.makedirs(os.path.dirname(WEIGHT_PATH),exist_ok=True)
    print(f"\n=========== FOLD {fold+1} ===========")

    train_df = df.iloc[train_idx]
    test_df  = df.iloc[test_idx]
    bs =32
    # ---- Loaders ----
    train_loader = DataSequenceLoader(train_df, batch_size=bs, shuffle=True)
    test_loader  = DataSequenceLoader(test_df, batch_size=bs, shuffle=False)

    # ---- Build Model ----
    model = build_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # ---- Train ----
    model.fit(train_loader,  validation_data=test_loader, epochs=100, verbose=1)

    model.save_weights(WEIGHT_PATH)
    # ---- Predict ----
    y_true =unseen_df['label'].values
    y_prob = model.predict(unseen_loader)
    y_pred = (y_prob > 0.5).astype(int)

    # ---- Confusion Matrix ----
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # ---- Metrics ----
    ACC  = accuracy_score(y_true, y_pred)
    SN   = recall_score(y_true, y_pred)
    SP   = tn / (tn + fp) if (tn + fp) > 0 else 0
    MCC  = matthews_corrcoef(y_true, y_pred)
    PREC = precision_score(y_true, y_pred)
    NPV  = tn / (tn + fn) if (tn + fn) > 0 else 0
    F1   = f1_score(y_true, y_pred)
    AUC  = roc_auc_score(y_true, y_prob)
    AUPR = average_precision_score(y_true, y_prob)

    fold_metrics = {
        "ACC": ACC,
        "SN": SN,
        "SP": SP,
        "MCC": MCC,
        "Precision": PREC,
        "NPV": NPV,
        "F1": F1,
        "AUC": AUC,
        "AUPRC": AUPR
    }

    fold_results.append(fold_metrics)

    # ---- Print fold results ----
    display(pd.DataFrame([fold_metrics]))

    # ---- Cleanup model + loaders ----
    del model
    del train_loader
    del test_loader
    gc.collect()


# ================================================================
# Final results: per fold + mean
# ================================================================
results_df = pd.DataFrame(fold_results)

print("\n=========== FINAL 5-FOLD RESULTS ===========")
display(results_df)

print("\n=========== MEAN METRICS ===========")
display(results_df.mean())



=========== FOLD 1 ===========


I0000 00:00:1764835330.435296    8819 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79078 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:81:00.0, compute capability: 8.0
/home/mhossai5/.conda/envs/knlp/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


I0000 00:00:1764835340.199419   13582 service.cc:148] XLA service 0x7ffe74001e90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764835340.200734   13582 service.cc:156]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
I0000 00:00:1764835341.304179   13582 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/622 ━━━━━━━━━━━━━━━━━━━━ 34:00:42 197s/step - accuracy: 0.5625 - loss: 2.3832

...

622/622 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.5182 - loss: 1.5910

622/622 ━━━━━━━━━━━━━━━━━━━━ 98s 138ms/step - accuracy: 0.5182 - loss: 1.5902 - val_accuracy: 0.6995 - val_loss: 0.6446
Epoch 2/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 76s 123ms/step - accuracy: 0.6077 - loss: 0.6633 - val_accuracy: 0.7156 - val_loss: 0.6040
Epoch 3/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 75s 121ms/step - accuracy: 0.6712 - loss: 0.6077 - val_accuracy: 0.7381 - val_loss: 0.5762
Epoch 4/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.7008 - loss: 0.5772 - val_accuracy: 0.7470 - val_loss: 0.5548
Epoch 5/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 75s 121ms/step - accuracy: 0.7178 - loss: 0.5636 - val_accuracy: 0.7510 - val_loss: 0.5498
Epoch 6/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 75s 121ms/step - accuracy: 0.7205 - loss: 0.5472 - val_accuracy: 0.7564 - val_loss: 0.5424
Epoch 7/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 76s 121ms/step - accuracy: 0.7349 - loss: 0.5347 - val_accuracy: 0.7522 - val_loss: 0.5321
Epoch 8/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 75s 121ms/step - accuracy: 0.7299 - loss: 0.534

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



622/622 ━━━━━━━━━━━━━━━━━━━━ 79s 127ms/step - accuracy: 0.8257 - loss: 0.3822 - val_accuracy: 0.8279 - val_loss: 0.4054
Epoch 51/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 77s 123ms/step - accuracy: 0.8236 - loss: 0.3900 - val_accuracy: 0.8273 - val_loss: 0.3958
Epoch 52/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 75s 121ms/step - accuracy: 0.8277 - loss: 0.3873 - val_accuracy: 0.8237 - val_loss: 0.3983
Epoch 53/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.8269 - loss: 0.3841 - val_accuracy: 0.8283 - val_loss: 0.3979
Epoch 54/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.8316 - loss: 0.3764 - val_accuracy: 0.8287 - val_loss: 0.3957
Epoch 55/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 77s 123ms/step - accuracy: 0.8214 - loss: 0.3917 - val_accuracy: 0.8331 - val_loss: 0.3879
Epoch 56/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.8345 - loss: 0.3687 - val_accuracy: 0.8335 - val_loss: 0.3890
Epoch 57/100
622/622 ━━━━━━━━━━━━━━━━━━━━ 75s 121ms/step - accuracy: 0.8287 - loss

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [7]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score, roc_auc_score,
    average_precision_score, matthews_corrcoef, confusion_matrix
)

# ================================================================
# Setup
# ================================================================
train_df,unseen_df = df[df['split']=='seen'].copy(),  df[df['split']=='unseen'].copy()

X = train_df.index.values  # DataSequenceLoader loads using df, so features = df itself
y = train_df['label'].values
bs =32
unseen_loader  = DataSequenceLoader(unseen_df, batch_size=bs, shuffle=False)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_results = []


# ================================================================
# 5-Fold Training Loop
# ================================================================
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    WEIGHT_PATH = f"../weights/LlamaCrossAttn_HIV-RLEAAI/model_fold_{fold}.weights.h5"
    os.makedirs(os.path.dirname(WEIGHT_PATH),exist_ok=True)
    print(f"\n=========== FOLD {fold+1} ===========")

    train_df = df.iloc[train_idx]
    test_df  = df.iloc[test_idx]
    bs =32
    # ---- Loaders ----
    train_loader = DataSequenceLoader(train_df, batch_size=bs, shuffle=True)
    test_loader  = DataSequenceLoader(test_df, batch_size=bs, shuffle=False)

    # ---- Build Model ----
    model = build_model()
    model.load_weights(WEIGHT_PATH)
    # ---- Predict ----
    y_true =unseen_df['label'].values
    y_prob = model.predict(unseen_loader)
    y_pred = (y_prob > 0.5).astype(int)

    # ---- Confusion Matrix ----
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # ---- Metrics ----
    ACC  = accuracy_score(y_true, y_pred)
    SN   = recall_score(y_true, y_pred)
    SP   = tn / (tn + fp) if (tn + fp) > 0 else 0
    MCC  = matthews_corrcoef(y_true, y_pred)
    PREC = precision_score(y_true, y_pred)
    NPV  = tn / (tn + fn) if (tn + fn) > 0 else 0
    F1   = f1_score(y_true, y_pred)
    AUC  = roc_auc_score(y_true, y_prob)
    AUPR = average_precision_score(y_true, y_prob)

    fold_metrics = {
        "ACC": ACC,
        "SN": SN,
        "SP": SP,
        "MCC": MCC,
        "Precision": PREC,
        "NPV": NPV,
        "F1": F1,
        "AUC": AUC,
        "AUPRC": AUPR
    }

    fold_results.append(fold_metrics)

    # ---- Print fold results ----
    display(pd.DataFrame([fold_metrics]))

    # ---- Cleanup model + loaders ----
    del model
    del train_loader
    del test_loader
    gc.collect()


# ================================================================
# Final results: per fold + mean
# ================================================================
results_df = pd.DataFrame(fold_results)

print("\n=========== FINAL 5-FOLD RESULTS ===========")
display(results_df)

print("\n=========== MEAN METRICS ===========")
display(results_df.mean())



=========== FOLD 1 ===========


I0000 00:00:1764881659.861414  118629 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79078 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:81:00.0, compute capability: 8.0
/home/mhossai5/.conda/envs/knlp/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1764881663.937401  118956 service.cc:148] XLA service 0x7ffe740029d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764881663.937476  118956 service.cc:156]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
I0000 00:00:1764881664.370173  118956 cuda_dnn.cc:529] Loaded cuDNN version

  3/143 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step    

I0000 00:00:1764881712.840687  118956 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


142/143 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step

...

,ACC,SN,SP,MCC,Precision,NPV,F1,AUC,AUPRC
0,0.874533,0.860594,0.885335,0.745233,0.853293,0.891245,0.856928,0.951662,0.937467



=========== FINAL 5-FOLD RESULTS ===========


,ACC,SN,SP,MCC,Precision,NPV,F1,AUC,AUPRC
0,0.868600,0.902365,0.842434,0.739229,0.816113,0.917587,0.857075,0.952730,0.939776
1,0.878488,0.821842,0.922387,0.752657,0.891376,0.869805,0.855198,0.953557,0.940023
2,0.878488,0.858078,0.894306,0.752861,0.862854,0.890485,0.860459,0.955961,0.942900
3,0.872555,0.874182,0.871295,0.742585,0.840348,0.899356,0.856931,0.950831,0.938654
4,0.874533,0.860594,0.885335,0.745233,0.853293,0.891245,0.856928,0.951662,0.937467



=========== MEAN METRICS ===========


ACC          0.874533
SN           0.863412
SP           0.883151
MCC          0.746513
Precision    0.852797
NPV          0.893696
F1           0.857318
AUC          0.952948
AUPRC        0.939764
dtype: float64